In [7]:
## Import Library
import os
import json
import copy
import re
import string
import openai

# from azure.core.credentials import AzureKeyCredential
# from azure.identity import AzureDeveloperCliCredential
# from azure.search.documents import SearchClient
# from azure.search.documents.indexes import SearchIndexClient
# from azure.search.documents.indexes.models import (
#     HnswParameters,
#     PrioritizedFields,
#     SearchableField,
#     SearchField,
#     SearchFieldDataType,
#     SearchIndex,
#     SemanticConfiguration,
#     SemanticField,
#     SemanticSettings,
#     SimpleField,
#     VectorSearch,
#     VectorSearchAlgorithmConfiguration,
# )
# from azure.storage.blob import BlobServiceClient
# from azure.storage.blob import BlobServiceClient
# from azure.core.exceptions import ResourceExistsError

# from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.vectorstores.azuresearch import AzureSearch
# from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.vectorstores.azuresearch import AzureSearch
# from langchain.chains import RetrievalQA
# from langchain.chains.question_answering import load_qa_chain
# #from langchain.retrievers import AzureCognitiveSearchRetriever
# from langdetect import detect
# from langchain.prompts import PromptTemplate

# # Create chain to answer questions
# from langchain.chains import RetrievalQA
# from langchain.chains.question_answering import load_qa_chain
# from langchain.memory import ConversationBufferMemory
# from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# # Import Azure OpenAI
# from langchain.llms import AzureOpenAI 
from langchain.chat_models import AzureChatOpenAI
# from langchain.schema import HumanMessage

# from langchain.chains import LLMChain
# from langchain.llms import AzureOpenAI 
# from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
# from langchain.chat_models import AzureChatOpenAI
# from langchain.memory import CosmosDBChatMessageHistory

# from langchain.chains import create_extraction_chain
# from langchain.output_parsers import StructuredOutputParser, ResponseSchema
# from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate

#import textwrap
#import logging


In [8]:
index_name = "credit-proposal"
search_service = "gptdemosearch"
search_api_key = "PcAZcXbX2hJsxMYExc2SnkMFO0D94p7Zw3Qzeu5WjYAzSeDMuR5O"
storage_service = "creditproposal"
storage_api_key = "hJ2qb//J1I1KmVeDHBpwEpnwluoJzm+b6puc5h7k+dnDSFQ0oxuh1qBz+qPB/ZT7gZvGufwRbUrN+ASto6JOCw=="
connect_str = f"DefaultEndpointsProtocol=https;AccountName={storage_service};AccountKey={storage_api_key}"

doc_intell_endpoint = "https://doc-intelligence-test.cognitiveservices.azure.com/"
doc_intell_key = "9fac3bb92b3c4ef292c20df9641c7374"

# set up openai environment
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_BASE"] = "https://pwcjay.openai.azure.com/"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
os.environ["OPENAI_API_KEY"] = "f282a661571f45a0bdfdcd295ac808e7"

os.environ["AZURE_COGNITIVE_SEARCH_SERVICE_NAME"] = search_service
os.environ["AZURE_COGNITIVE_SEARCH_API_KEY"] = search_api_key
os.environ["AZURE_INDEX_NAME"] = index_name

In [9]:
def load_json(json_path):
    with open(json_path, "r" ,encoding="utf-8") as f:
        data = json.load(f)
    return data


#This funcition is to extract RM notes for web, call by app.py
def extract_rm_notes(rm_note_txt):

    # read config files in config directory
    hierarchy_file_name = "config/hierarchy.json"

    hierarchy_dict_list = load_json(hierarchy_file_name)

    prompt_template_for_extracting_rm_note = """
    Read the RM note (Keyword: rm_note) to answer the input question (Keyword: Question):
    
    rm_note (Keyword: rm_note):
    {rm_note}
    ======
    Question (Keyword: Question):
    {question}
    ======
    
    Then answer the question based on the above aggregrate context

    Guidance when you do not have that information:
    1. When you don't have the specific information based on rm_note (Keyword: rm_note), you have to write it in the following format: [Information not available]
    2. You must not create the information by yourself if you don't have relevant information
    3. You cannot say "It's unclear that", please refer to point 1 for the formatting for requesting further information

    Rules you have to follow:
    1. Please provide your answer in English
    2. do not start your answer with "Based on the given information"
    3. The example (Keyword: proposal_example) above is just for your reference only to improve your theme, you must not directly copy the content in the examples
    4. If possible, try to expand the information provided from the RM
    5. Do not create any figures by make-up 

    Take a deep breath and work on this step by step
    """
    
    rm_prompt_template = PromptTemplate(template=prompt_template_for_extracting_rm_note, input_variables=["rm_note", "question"])

    llm = AzureChatOpenAI(deployment_name="gpt-35-16k", temperature=0.1,
                            openai_api_version="2023-05-15", openai_api_base="https://pwcjay.openai.azure.com/")


    output_dict_list = []
    for dictionary in hierarchy_dict_list:
        chain = LLMChain(llm=llm, prompt=rm_prompt_template)
        dictionary["Value"] = chain({"rm_note":rm_note_txt, "question": dictionary["Question"]})['text']
        dictionary["Value"] = dictionary["Value"].replace("Based on the given information, ", "")
        output_dict_list.append(dictionary)
    # json.dump(output_dict_list, open(output_json_name, "w"), indent=4)

    return output_dict_list

    # return hierarchy_dict_list, rm_prompt_template, llm_rm_note, rm_note_txt

In [10]:
rm_note_txt = """Client: Gogovan
Industry: Logistics and Delivery Services
Date: 20 Nov 2023

Meeting Summary:	
Today, I evaluated Gogovan's credit proposal for their expansion plans, technology investments, and working capital needs.	

Client Background:	
Gogovan is a leading logistics and delivery service provider, offering on-demand delivery solutions to individuals and businesses. Established in 2013, the company has rapidly expanded its operations and established a strong presence in the market. Gogovan operates a user-friendly mobile application and web platform, connecting customers with a network of professional drivers and delivery partners.	

Financial Performance:	
Gogovan has demonstrated consistent revenue growth over the past few years, driven by increasing customer adoption and expansion into new markets. The company's financial statements reflect a healthy profitability margin, indicating effective cost management and operational efficiency. Cash flow from operations has been positive, providing a stable source of funds to support working capital requirements and ongoing business operations.	

Market Position and Competitive Landscape:	
Gogovan has successfully positioned itself as a market leader in the logistics and delivery industry, leveraging its strong brand recognition and innovative technology platform. The company has built a robust network of drivers and delivery partners, enabling quick and reliable service fulfillment. Gogovan's competitive advantage lies in its ability to offer cost-effective and flexible solutions tailored to meet the needs of various customer segments, including e-commerce, retail, and individual users.	

Growth Strategy and Market Potential:	
Gogovan has outlined a comprehensive growth strategy focused on expanding its geographical presence, diversifying its service offerings, and enhancing customer experience. The company plans to enter new markets, both domestically and internationally, to capture additional customer segments and increase market share. Gogovan aims to invest in technology and infrastructure improvements to streamline operations, optimize delivery routes, and enhance overall efficiency.	

Risk Assessment:	
The logistics industry is subject to various risks, including intense competition, regulatory changes, and economic downturns. Gogovan has implemented risk mitigation measures such as diversification of services and markets, maintaining strong relationships with key partners, and closely monitoring market trends. Operational risks, such as driver availability, vehicle maintenance, and service disruptions, are managed through rigorous driver screening, continuous training programs, and proactive maintenance schedules. Financial risks are mitigated by maintaining a healthy liquidity position, diversifying funding sources, and prudent financial management practices.	

Credit Request and Repayment Plan:	
Gogovan is requesting a credit facility of $10million to support its expansion plans, technology investments, and working capital needs. The proposed repayment plan consists of regular principal and interest payments over a 3 years term, aligning with the company's projected cash flow generation and financial performance.	

Conclusion:	
Gogovan has demonstrated a strong market position, consistent financial performance, and a well-defined growth strategy. With its robust operational capabilities, innovative technology platform, and customer-centric approach, the company is well-positioned to capitalize on the growing demand for logistics and delivery services. The proposed credit facility, in line with the company's financial projections, will support Gogovan's expansion plans and enable it to maintain its competitive edge in the market.	
"""

In [11]:
extract_rm_notes(rm_note_txt)

[{'Section': 'A. Project Overview',
  'Sub Section': 'Client Name',
  'Breakdown': '',
  'Value': 'Gogovan',
  'Question': 'what is the company/Corporation name? Please just return the name, if RM notes Client: () have vaule,  return the Client name. else, serach from relevent documents.',
  'ID': 0},
 {'Section': 'A. Project Overview',
  'Sub Section': 'Project Name',
  'Breakdown': '',
  'Value': 'The company/corporation project name is Gogovan.',
  'Question': 'What is the company/Corporation Project Name?',
  'ID': 1},
 {'Section': 'A. Project Overview',
  'Sub Section': 'Project Description',
  'Breakdown': '',
  'Value': 'The project description for Gogovan is to expand its geographical presence, diversify its service offerings, and enhance customer experience. This includes entering new markets, both domestically and internationally, to capture additional customer segments and increase market share. Gogovan also plans to invest in technology and infrastructure improvements to st